In [1]:
import numpy as np
import torch
from torchvision import transforms, models, datasets
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

In [2]:
device =torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [3]:

root = "./images/patches"
transformation = transforms.Compose([transforms.Resize((224,224)),
                                     transforms.ToTensor()
                                    ])
data = datasets.ImageFolder(root = root, transform=transformation)
train_set, test_set = torch.utils.data.random_split(data, [int(len(data) * 0.8), len(data) -int(len(data) * 0.8) ])

#csvloader = torch.utils.data.DataLoader(data, batch_size = 1, shuffle = True)
# snow, nosnow = [], []
# for img, lbl in csvloader:
#     if(lbl == 1):
#         snow.append(img)
# snow = torch.cat(snow, dim = 0)

# train_snow, test_snow = torch.split(snow, [int(0.8*len(snow)), len(snow) - int(0.8*len(snow))])
# print('data loaded')
# for num,part in enumerate(torch.split(train_snow,10)):
#     torch.save(part, f'train_patches_snow{num}.pt')
# for num,part in enumerate(torch.split(test_snow,10)):
#     torch.save(part, f'test_patches_snow{num}.pt')

# for img, lbl in csvloader:
#     if(lbl == 0):
#         nosnow.append(img)
# nosnow = torch.cat(nosnow, dim = 0)

# train_nosnow, test_nosnow = torch.split(nosnow, [int(0.8*len(nosnow)), len(nosnow) - int(0.8*len(nosnow))])
# print('data loaded')

# for num,part in enumerate(torch.split(train_nosnow,10)):
#     torch.save(part, f'train_patches_nosnow{num}.pt')
# for num,part in enumerate(torch.split(test_nosnow,10)):
#     torch.save(part, f'test_patches_nosnow{num}.pt')




In [4]:
# transformation = transforms.Compose([transforms.RandomPerspective(0.6, p=0.5),transforms.GaussianBlur(5)])

# class riverdataset(torch.utils.data.Dataset):
#     def __init__(self, snow_file, nosnow_file, transform = None):
#         """
#         Arguments:
#             snow_file (string): Path to the .pt file
#             nosnow_file (string): Path to the .pt file
#             noise (boolean): add guassian noise
#         """
#         snow = torch.load(snow_file)
#         nosnow = torch.load(nosnow_file)
#         labels1 = torch.tensor([1 for i in range(len(snow))])
#         labels0 = torch.tensor([0 for i in range(len(nosnow))])
#         self.data = torch.cat([snow, nosnow], dim= 0)
#         self.labels = torch.cat([labels1, labels0], dim = 0)
        
#         self.transform = transform

#     def __len__(self):
#         return len(self.labels)
    
#     def __getitem__(self, idx):
#         image = self.data[idx]
#         if(self.transform):
#             image = self.transform(image)
#         return image, self.labels[idx]
    

# train_set = riverdataset("train_patches_snow.pt", "train_patches_nosnow.pt", transformation)
# test_set = riverdataset('test_patches_snow.pt', "test_patches_nosnow.pt", False)


In [5]:
#Create additional layers for snow classification
model = models.resnet50(weights =models.ResNet50_Weights.IMAGENET1K_V2)
for param in model.parameters():
    param.requires_grad = False   

model.fc = torch.nn.Sequential(
    torch.nn.Linear(2048, 128),
    torch.nn.ReLU(),
    torch.nn.Linear(128,2),
    torch.nn.Softmax(dim= 1)
)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
#parameters

lr = 0.0001
batch_size = 256
epochs =40
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr)
train_loader = torch.utils.data.DataLoader(train_set, batch_size= batch_size, shuffle= True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size= 1, shuffle= False)

In [7]:
def calc_test_accuracy(model, dataloader,epoch, save_pic = False, criterion =torch.nn.CrossEntropyLoss()):
      model.eval()
      TP = 0
      FP = 0
      TN = 0
      FN = 0
      val_loss = 0
      with torch.no_grad():
            for num, (i, l) in enumerate(dataloader):
                  if(save_pic):     
                        plt.imsave(f"image{num}.png",i.squeeze(0).permute(1,2,0).numpy() )
                 
                  if(l == torch.tensor([0])):
                        l = torch.tensor([1, 0])
                  else:
                        l = torch.tensor([0, 1])
                  l = l.reshape([1,2])
                  i,l = i.to(device), l.to(device).float()
                  t_out = model(i)
                  
                  
                  val_loss+= criterion(t_out, l).item()
                  predicted = np.argmax(t_out.cpu().numpy(), axis = 1)
                  true_label = np.argmax(l.cpu().numpy(), axis =1)
                 
                  if(predicted == 1 and true_label == 1):
                        TP +=1
                  elif(predicted == 0 and true_label == 0):
                        TN +=1
                  if(predicted == 1 and true_label == 0):
                        FP +=1
                        #plt.imsave(f"imageFP{num}.png",i.cpu().squeeze(0).permute(1,2,0).numpy() )
                  elif(predicted == 0 and true_label == 1):
                        FN +=1
                        #plt.imsave(f"imageFN{num}.png",i.cpu().squeeze(0).permute(1,2,0).numpy() )
            try:
                  precision = TP/(TP + FP)
            except:
                  precision = 0
            try:
                  recall = TP/(TP + FN)
            except:
                  recall = 0
            stats = dict(
                  epoch = epoch,
                  accuracy = (TN+TP) /(TP+TN+FP+FN),
                  precision = precision,
                  recall = recall,
                  loss = val_loss/(TP+FP+TN+FN)
            
            )
            # print(f'True positives: {TP}')
            # print(f'True negatives: {TN}')
            # print(f'False positives: {FP}')
            
            # print(f'False negatives: {FN}')
            return stats


In [8]:

#training loop
running_loss = 0 
train_loss, test_loss = [],[]
# model = torch.load("model.pth")
model.train()
verbose = 10
steps = 0

for epoch in range(0, epochs):
    print(epoch)
    for images,labels in train_loader:
        steps +=1
        if(steps %10 == 0):
            print(f'step : {steps}/{len(train_loader)}')
        labels = torch.nn.functional.one_hot(labels)
        images,labels = images.to(device), labels.to(device).float()
        optimizer.zero_grad()
        out = model(images)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    steps  = 0 
        
    val_loss = 0
    num_correct = 0
    model.eval()
    train_loss.append(running_loss/len(train_loader))
    if(epoch % 2 == 0):
        with torch.no_grad():
            print('testing')
            stats = calc_test_accuracy(model, test_loader, epoch+1)
            
            test_loss.append(stats["loss"])
            stats["training_loss"] = train_loss
            torch.save(stats, f"stats-{epoch+1}.pt")
            print(stats)
    running_loss= 0
    model.train()
    torch.save(model, 'model.pth')
    
  


        
    

0
step : 10/178
step : 20/178
step : 30/178
step : 40/178
step : 50/178
step : 60/178
step : 70/178
step : 80/178
step : 90/178
step : 100/178
step : 110/178
step : 120/178
step : 130/178
step : 140/178
step : 150/178
step : 160/178
step : 170/178
testing
{'epoch': 1, 'accuracy': 0.8807307219392236, 'precision': 0.8818459328425211, 'recall': 0.875245054357512, 'loss': 0.4496864829334793, 'training_loss': [0.5187478770365875]}
1
step : 10/178
step : 20/178
step : 30/178
step : 40/178
step : 50/178
step : 60/178
step : 70/178
step : 80/178
step : 90/178
step : 100/178
step : 110/178
step : 120/178
step : 130/178
step : 140/178
step : 150/178
step : 160/178
step : 170/178
2
step : 10/178
step : 20/178
step : 30/178
step : 40/178
step : 50/178
step : 60/178
step : 70/178
step : 80/178
step : 90/178
step : 100/178
step : 110/178
step : 120/178
step : 130/178
step : 140/178
step : 150/178
step : 160/178
step : 170/178
testing
{'epoch': 3, 'accuracy': 0.91392938696645, 'precision': 0.93862473

In [ ]:
model = torch.load('model.pth')
stats = calc_test_accuracy(model, test_loader, epoch+1)
stats["training_loss"] = train_loss
torch.save(stats, f"stats-{epoch+1}.pt")
print(stats)



In [ ]:
fig, ax = plt.subplots()

# Plot the data
ax.plot(train_loss, marker='o', linestyle='-', color='b')

# Customize the plot
ax.set_title('Snow ResNet50 Training Loss')
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')

# Display the plot
plt.show()

In [ ]:
fig, ax = plt.subplots()

# Plot the data
ax.plot(test_loss,marker='o', linestyle='-', color='b')

# Customize the plot
ax.set_title('Snow classifier ResNet50 Test Loss')
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')

# Display the plot
plt.show()

In [ ]:
accuracies = []
fig, ax = plt.subplots()
for i in range(1,21):
    accuracies.append(torch.load(f'stats-{i}.pt')['accuracy'])
# Plot the data
print(accuracies)
ax.plot(accuracies,marker='o', linestyle='-', color='b')

# Customize the plot
ax.set_title('Snow Classifier ResNet50 Accuracy')
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')

# Display the plot
plt.show()

